# Generating, Indexing and Searching Embeddings

## Installing the Package

In [1]:
%cd ../../../
!python3 -m pip install --upgrade .

/home/gpadmin/GreenplumPython
Defaulting to user installation because normal site-packages is not writeable
Processing /home/gpadmin/GreenplumPython
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for greenplum-python: filename=greenplum_python-1.0.1-py3-none-any.whl size=70618 sha256=22e5718895157b66b4e6a4dd9bbb7164f4039f4d59dbf58fb519da409b246e38
  Stored in directory: /tmp/pip-ephem-wheel-cache-q9fmewst/wheels/bb/1f/99/ff8594e48ec11df99af6e0ee8611a5e560e9f44d1a3fefb351
Successfully built greenplum-python


## Preparing Data

In [2]:
content = ["I have a dog.", "I like eating apples."]

import greenplumpython as gp

db = gp.database("postgresql://localhost:7000")
t = (
    db.create_dataframe(columns={"id": range(len(content)), "content": content})
    .save_as(
        table_name="text_sample",
        column_names=["id", "content"],
        distribution_key={"id"},
        distribution_type="hash",
    )
    .check_unique(columns={"id"})
)

## Generating and Indexing Embeddings

In [3]:
import greenplumpython.experimental.embedding

t = t.embedding().create_index(column="content", model="all-MiniLM-L6-v2")
t

id,content
0,I have a dog.
1,I like eating apples.


## Searching Embeddings

In [4]:
t.embedding().search(column="content", query="apple", top_k=1)

id,content
1,I like eating apples.


## Cleaning All at Once

In [6]:
%reload_ext sql
%sql postgresql://localhost:7000
%sql DROP TABLE text_sample CASCADE;

 * postgresql://localhost:7000
Done.


[]